In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Step1: Necessery imports
import pandas as pd
import numpy as np
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Step2: Loading the data 
#Change the csv file for another brand
#Added Data types for memory optimization and faster loading
data = pd.read_csv('/kaggle/input/most-searched-products-2019-vs-2020/Most_Searched_Products_2019_2020.csv')
            
print(data.info()) 

# INTRO: World maps exploring the pre and post-pandemic demand for popular products

Throughout the COVID-19 pandemic, consumers worldwide have realized large behavioral changes related to their day-to-day lives — exceptional the way they buy online.

The Wall Street Journal quoted analysts’ estimations that the e-commerce sector rose by approximately 50 percent through 2020 only. Indeed, when analyzing just the United States, online purchases through the 2020 holiday period increased by a tremendous 32.2 percent compared to the same period in 2019.

Consumers spend a lot of time at home and exhibit their rising preference for online buying. The way buyers search for products online has evolved. According to Google, Internet searches for cookware grown by 53 percent from 2019 to 2020, and searches for streaming devices rose by 31 percent over the same period. Search interest for coffee makers climbed to an all-time high in 2020, growing by more than 33 percent from 2019.

We can take unusual approach to analyze and finally visualize worldwide eCommerce trends from 2019 to 2020. We examine historical Google search volume data of approximately six hundred popular products across 189 nations globally to define the products that were searched at a disproportionately high frequency to find which country “Googled” the most popular consumer products at the highest rate.

The world maps below compare historical exploration volume data for the buyer products referenced above for 2019 and 2020.

# Highlights from the data:
From 2019 to 2020, there was a 51% increase in the number of countries where the most-searched product was an iPhone. There was an even sharper increase in countries searching for iPhones more than any other product during the 2020 holiday shopping season (a 71% increase compared to the previous year).
Throughout the holiday season in 2020, there was a 27% increase in the total number of countries that had a streaming device as their most searched product. In 2019, Amazon’s Firestick held the top streaming device spot in said countries, while Google’s Chromecast was the streaming device of choice for a holiday 2020.
From the holiday season in 2019 to 2020, there was a 157% increase in the total number of countries where a video game console was the most-searched product. The Nintendo Switch was the top gaming system in both 2019 and 2020.

In [ ]:
!pip install geopy==1.22.0
import math
import geopandas as gpd
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from geopandas.tools import geocode
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

In [ ]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

data[['Latitude', 'Longitude', 'geometry']] = data.apply(lambda x: my_geocoder(x['Country']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

# Drop data that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(data, geometry=data.geometry)
data.crs = {'init': 'epsg:4326'}
data.head()

# Most Searched Products by Country 2019

In [ ]:
# Create a map
map = folium.Map(location=[50, 10], tiles='openstreetmap', zoom_start=6)

# Add points to the map
for idx, row in data.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Product2019']).add_to(map)

# Display the map
map

# Most Searched Products by Country 2020

In [ ]:
# Create a map
map = folium.Map(location=[50, 10], tiles='openstreetmap', zoom_start=6)

# Add points to the map
for idx, row in data.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Product2020']).add_to(map)

# Display the map
map

In [ ]:
data_model=data['Product2020'].value_counts().reset_index().rename(columns={'index':'Product2020','Product2020':'Count'})
data_model
fig = px.pie(data_model, values='Count', names='Product2020')

fig.update_traces(hoverinfo='label+percent', textinfo='value+percent', textfont_size=12,insidetextorientation='radial')
fig.update_layout(title="Product2020 Distribution",title_x=0.5)
fig.show()

In [ ]:
data_model=data['Product2019'].value_counts().reset_index().rename(columns={'index':'Product2019','Product2019':'Count'})
data_model
fig = px.pie(data_model, values='Count', names='Product2019')

fig.update_traces(hoverinfo='label+percent', textinfo='value+percent', textfont_size=12,insidetextorientation='radial')
fig.update_layout(title="Product2019 Distribution",title_x=0.5)
fig.show()